In [16]:
from data import generate_dataframe

df = generate_dataframe(["Musical_Instruments_5.json"])

Loading data from: ['Musical_Instruments_5.json']

filename             samples
Musical_Instruments_5 231392 

Data loaded, 231392 total samples.


In [17]:
from preprocessing import preprocess_samples
from utils import get_product_reviews

raw_corpus_samples = list(filter(lambda x: isinstance(x, str), df['reviewText']))
corpus_samples = preprocess_samples(raw_corpus_samples)

Applying lowercase


231344it [00:00, 2641574.89it/s]


Applying remove_punctuation


231344it [00:01, 135578.18it/s]


Applying lemmatize


231344it [10:25, 369.97it/s] 


In [19]:
product_id = 'B0002E3CK4'
raw_product_samples = get_product_reviews(df, product_id)
product_samples = preprocess_samples(raw_product_samples)

Applying lowercase


1879it [00:00, 2340688.21it/s]


Applying remove_punctuation


1879it [00:00, 197749.22it/s]


Applying lemmatize


1879it [00:03, 579.41it/s]


In [14]:
import n_grams
bigrams = n_grams.get_bigrams(product_samples, 10, 6)
trigrams = n_grams.get_trigrams(product_samples, 2, 17)
product_samples = n_grams.replace_n_grams(product_samples, bigrams, trigrams)

print(bigrams[:10])
print(trigrams[:10])

['clap along', 'pharrell williams', "i'm clap"]
['like curtis mayfield', "can't nothing bring"]


In [15]:
from nltk import word_tokenize
import train
tf_vectorizer = train.get_tf_vectorizer()
tf_vectorizer.set_params(tokenizer=word_tokenize)

Loading pretrained vectorizer...


CountVectorizer(max_df=0.01, max_features=1000, min_df=2, stop_words='english',
                tokenizer=<function word_tokenize at 0x16b1ad940>)

In [20]:
product_documents = tf_vectorizer.fit_transform(product_samples)

In [ ]:
import evaluation_metrics as em
freqs = em.get_word_freqs(product_samples)
bi_freqs = em.get_bi_freqs(product_samples)

 83%|███████████████████████████████████████████████████████████████████████████████████████▎                 | 1562/1879 [17:20<15:20,  2.90s/it]

In [ ]:
import train
# Compute topic coherence
n_components = range(1,10)
coherences = []
num_docs = len(product_samples)

for num in n_components:
  lda = train.get_lda(samples=product_documents, n_components=num)
  tf_feature_names = tf_vectorizer.get_feature_names_out()
  coherence = em.avg_umass(lda, tf_feature_names, train.n_top_words, freqs, bi_freqs, num_docs)[1]
  coherences.append(coherence)

for i, c in enumerate(coherences):
  print("Number of topics: ", n_components[i])
  print("Average UMass coherence: ", c)
  print()


In [ ]:
from utils import plot_top_words
coherences = [abs(c) for c in coherences]
n_components = n_components[coherences.index(min(coherences))]

lda = train.get_lda(samples=product_documents, n_components=n_components)

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, train.n_top_words, "Topics in LDA model")